In [ ]:
import itertools

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

In [ ]:
# Graph generation

def _get_first_example_graph():
    G = nx.DiGraph()
    G.add_edges_from([
        (1, 3), (2, 3),
        (3, 4), (3, 8),
        (4, 6), (5, 6),
        (6, 7)
    ])
    
    node_pos = {
        1: (0, 0), 2: (2, 0), 
        3: (1, -1), 
        5: (-2, -2), 4: (0, -2), 8: (2, -2),
        6: (-1, -3), 7: (-1, -4)
    }
    
    return G, node_pos

def get_graph(which):
    if 'first' in which:
        return _get_first_example_graph()
    elif 'second' in which:
        res = _get_first_example_graph()
        res[0].add_edge(1, 5)
        return res
    elif 'random' in which:
        _g = nx.fast_gnp_random_graph(10, 0.2, directed=True)
        g = nx.DiGraph([(u, v) for u, v in _g.edges() if u < v])
            
        return g, None
    else:
        raise RuntimeError('Unknown graph')


In [ ]:
# Visualization
def visualize(G, C, node_pos):
    colors = ['gray' if v in C else 'red' for v in G.nodes()]
    nx.draw(G, with_labels=True, node_color=colors, pos=node_pos)

In [ ]:
# Helper functions
def _is_head_head(G, a, v, b):
    return G.has_edge(a, v) and G.has_edge(b, v)


def _is_head_tail(G, a, v, b):
    return G.has_edge(a, v) and G.has_edge(v, b)


def _is_tail_tail(G, a, v, b):
    return G.has_edge(v, a) and G.has_edge(v, b)


def is_sublist(x, of):
    return str(of)[1:-1].find(str(x)[1:-1]) >= 0


def get_all_node_pairs(G):
    return [(v1, v2) for v1, v2 in sorted(itertools.permutations(G.nodes(), r=2))]


def get_all_paths(G, v1, v2):
    return list(nx.all_simple_paths(nx.Graph(G), v1, v2))


In [ ]:
# Blocking paths

def get_blocking_paths(G, C):
    blocking_paths = []
    
    for a, v, b in sorted(itertools.permutations(G.nodes(), r=3)):
        if (_is_head_tail(G, a, v, b) or _is_head_tail(G, b, v, a)) and v in C:
            blocking_paths.append((a, v, b))
            continue
        
        if _is_tail_tail(G, a, v, b) and v in C:
            blocking_paths.append((a, v, b))
            continue
        
        if _is_head_head(G, a, v, b) and v not in C \
           and all(vv not in C for vv in nx.descendants(G, v)):
            blocking_paths.append((a, v, b))

    return blocking_paths


In [ ]:
# D-separability test

def is_d_separable(G, v1, v2, C):
    result = True
    blocking_paths = get_blocking_paths(G, C)
  
    for path in get_all_paths(G, v1, v2):
        if any(is_sublist(bp, of=path) for bp in blocking_paths):
            continue
    
        result = False
        break
        
    return result


def check_d_separability_for_all_node_pairs(G, C):
    df = pd.DataFrame(columns=['v1', 'v2', 'd_sep'])

    for v1, v2 in get_all_node_pairs(G):
        if G.has_edge(v1, v2) or G.has_edge(v2, v1):
            d_sep = '-'
        elif v1 in C or v2 in C:
            d_sep = '-'
        else:
            d_sep = is_d_separable(G, v1, v2, C)
        
        df = df.append({'v1': v1, 'v2': v2, 'd_sep': d_sep}, ignore_index=True)
        
    return df


In [ ]:
graph, node_pos = get_graph('first_example')
conditioning_set = set([7, ])
visualize(G=graph, C=conditioning_set, node_pos=node_pos)
df = check_d_separability_for_all_node_pairs(graph, conditioning_set)

In [ ]:
df[df.d_sep == True]

In [ ]:
df